In [40]:
plt.close('all')

In [3]:
import datetime
import sys

from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen

import pandas as pd
from pandas_datareader import data
from itertools import islice

from scipy.signal import savgol_filter
import statsmodels
import statsmodels.api as sm
import scipy.stats as stats
import scipy.optimize as optimize
import numpy as np
#from mpl_finance import candlestick_ohlc
import matplotlib.pyplot as plt
#For the usual plots!
#matplotlib inline
#Makes graphs in jupyter interactive!
%matplotlib widget

tickers = ['SPY', 'AAPL', 'MSFT', 'PYPL', 'ISRG', 'GM', 'KODK', 'INTC', 'PANW']

#Removing warnings
import warnings
warnings.filterwarnings('ignore')

In [9]:
plt.close('all')

def stock_data(startd, endd, tickers):
    df = data.DataReader(tickers, 'yahoo', startd, endd).reset_index()
#df.set_index('Date', inplace=True)            <-- weggehaald zodat polyfit en backtest "i-1" werken
#uitrekenen van de daily return (cert)
    for ticker in tickers:
        df[('Return', ticker)] = df[('Close', ticker)].shift(1) - df[('Close', ticker)] 
    for ticker in tickers:
        df[('Percent_return', ticker)] = df[('Close', ticker)].pct_change()
    return df

def smoothing_sma_savgol(SMA, tickers):
    x = SMA.replace('SMA', '')
    x = int(x)
    for t in tickers:
        savgol_len = len(dfsma[(SMA + '_change', t)].dropna())
        if (savgol_len % 2) == 0:
            savgol_len += 1
            if (savgol_len % 2) != 0:
                return savgol_len
            else:
                continue
    #No real rules for polyorder, just use what fits best for its purpose!
        savgol = savgol_filter(dfsma[(SMA + '_change', t)].dropna(), savgol_len, polyorder=100)
        insert_len =  len(dfsma[(SMA + '_change', t)]) - len(savgol)
        savgol = np.insert(savgol, 0, np.zeros(insert_len) + np.nan)
        savgol = pd.Series(savgol, index=dfsma.index)
        dfsma[(SMA + '_change_smooth_savgol', t)] = savgol
    
def polyfit_df(tickers, df):
    dftemp = df.copy()
    for t in tickers:
        result = np.polyfit(dftemp[('Percent_return', t)].dropna().index, dftemp[('Percent_return', t)].dropna(), 50)
        poly_eq = np.poly1d(result)
        dftemp[('Polyfit', t)] = poly_eq(dftemp.index)
    dftemp = dftemp.reset_index()
    return dftemp

def smoothing_sma_sma(SMA, tickers):
    x = SMA.replace('SMA', '')
    x = int(x)
    for t in tickers:
        dfsma[(SMA + '_change_smooth_sma', t)] = dfsma[(SMA + '_change', t)].rolling(x).mean()
        
def smoothing_sma_ema(SMA, tickers):
    x = SMA.replace('SMA', '')
    x = int(x)
    for t in tickers:
        dfsma[(SMA + '_change_smooth_ema', t)] = dfsma[(SMA + '_change', t)].ewm(span=x, adjust=False).mean()

# StopLoss ATR 

In [39]:
start_time = datetime.datetime.now().time()
print(start_time)

tickers = ['SPY', 'AAPL', 'MSFT', 'PYPL', 'ISRG', 'GM', 'INTC', 'PANW']
tickers = ['SPY']
startd = datetime.datetime(2016, 1, 1) 
#endd_1 = datetime.datetime(2019, 1, 1)
endd_2 = datetime.datetime(2020, 1, 1)
df = data.DataReader(tickers, 'yahoo', startd, endd_2).reset_index()
for t in tickers:
    df[('Percent_return', t)] = df[('Close', t)].pct_change()
    df[('Percent_return', 'Total')] = df[('Close', t)].pct_change()
print(df.index.max())
### If Close is 2xATR lager, SELL 3 dagen lang;   Meest basic implementatie van stoploss! 

#Functions#
def wwma(values, n):
    return values.ewm(alpha=1/n, adjust=False).mean()

def ATR(data, t , n):
    high = data[('High', t)]
    low = data[('Low', t)]
    close = data[('Close', t)]
    data[('Tr0', t)] = abs(high - low)
    data[('Tr1', t)] = abs(high - close.shift())
    data[('Tr2', t)] = abs(low - close.shift())
    Tr = data[[('Tr0', t), ('Tr1', t), ('Tr2', t)]].max(axis=1)
    Atr = wwma(Tr, n)
    data[('Atr', t)] = Atr
#De Stoploss 2xATR implementatie:
    Stoploss = data[('Close', t)] - (data[('Atr', t)] * 2)
    return Stoploss 
#End Function#

for t in tickers:
    df[('Percent_return', t)] = df[('Close', t)].pct_change()
    
#   Do-untill code:  https://stackoverflow.com/questions/1662161/is-there-a-do-until-in-python

df[('Stoploss', 'SPY')] = 0
x=0
for t in tickers:
    loop = iter(range(len(df)))
    df[('ATR', t)] = ATR(df, t, 1)
    for i in loop:
        df.loc[i, ('Stoploss', t)] = df.loc[x:i, ('ATR', t)].max()
        if df.loc[i, ('Stoploss', t)] >= df.loc[i, ('Close', t)]:
            df.loc[i+1:i+3, ('Strat', t)] = 0
            x=i+3
            next(islice(loop, 2, 2), None)
        if df.loc[i, ('Stoploss', t)] < df.loc[i, ('Close', t)]:
            df.loc[i+1, ('Strat', t)] = 1

###

df[('Strat_return', 'Total')] = 0
for t in tickers:
    df[('Strat_return', t)] = df[('Strat', t)] * df[('Percent_return', t)]
    df[('Strat_return', 'Total')] += df[('Strat_return', t)].cumsum()
df = df.dropna()
df = df.reset_index()
for t in tickers:
    fig, ax = plt.subplots(3, sharex=True, figsize=(10,7), gridspec_kw={'height_ratios':[2.5, 0.5, 1.5], 'hspace':0.02})
    ax[0].set_title(t)
    #ax[0].plot(df[('Percent_return', t)], linewidth=0.5)
    ax[0].plot(df[('Close', t)], linewidth=1.5)
    ax[0].plot(df[('Stoploss', t)], linewidth=1.5)
    ax[1].plot(df[('Strat', t)], linewidth=0.5)
    ax[2].plot(df[('Percent_return', 'Total')].cumsum(), linewidth=1)
    ax[2].plot(df[('Strat_return', t)].cumsum(), linewidth=1, color='r')
    #ax[0].plot(df[('Strat_return', t)].cumsum(), linewidth=1, color='r')
r1=df[('Strat_return', tickers[0])].sum()
r2=df[('Percent_return', tickers[0])].sum()
print(f'Strat return SPY = {r1}')
print(f'Hold return SPY = {r2}')
end_time = datetime.datetime.now().time()
print(f'Time when finished:                   {end_time}\n')
print(df.index.max())

01:22:04.828441
1005


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Strat return SPY = 0.4258268822144672
Hold return SPY = 0.5041826175961925
Time when finished:                   01:22:13.302941

1004


# StopLoss #1: Basic - 2% Decrease, 3 days lock-out

In [38]:
start_time = datetime.datetime.now().time()
print(start_time)

tickers = ['SPY', 'AAPL', 'MSFT', 'PYPL', 'ISRG', 'GM', 'INTC', 'PANW']
tickers = ['SPY']
startd = datetime.datetime(2016, 1, 1) 
#endd_1 = datetime.datetime(2019, 1, 1)
endd_2 = datetime.datetime(2020, 1, 1)
df = data.DataReader(tickers, 'yahoo', startd, endd_2).reset_index()
for t in tickers:
    df[('Percent_return', t)] = df[('Close', t)].pct_change()
    df[('Percent_return', 'Total')] = df[('Close', t)].pct_change()
print(df.index.max())

### If Low is 2% lager dan High, SELL 3 dagen lang;   Meest basic implementatie van stoploss!   

df[('Stoploss', 'SPY')] = 0
x=0
for t in tickers:
    loop = iter(range(len(df)))
    for i in loop:
        df.loc[i, ('Stoploss', t)] = df.loc[x:i, ('High', t)].max() * 0.98
        if df.loc[i, ('Stoploss', t)] >= df.loc[i, ('Low', t)] and df.loc[i-1, ('Percent_return', t)] >= 0:
            df.loc[i+1:i+3, ('Strat', t)] = 0
#index-overslaan fixed: https://stackoverflow.com/questions/17837316/how-do-i-skip-a-few-iterations-in-a-for-loop/17837409
#while-loop kan je een index skippen d.m.v. i+=3 ofzo
#sla 1 minder over dan bij x wordt opgetelt, anders is er een index skip van 0 -> 1
            next(islice(loop, 2, 2), None)
            x=i
        else:
            df.loc[i+1, ('Strat', t)] = 1


###

df[('Strat_return', 'Total')] = 0
for t in tickers:
    df[('Strat_return', t)] = df[('Strat', t)] * df[('Percent_return', t)]
    df[('Strat_return', 'Total')] += df[('Strat_return', t)].cumsum()
df = df.dropna()
df = df.reset_index()
for t in tickers:
    fig, ax = plt.subplots(3, sharex=True, figsize=(10,7), gridspec_kw={'height_ratios':[2.5, 0.5, 1.5], 'hspace':0.02})
    ax[0].set_title(t)
    #ax[0].plot(df[('Percent_return', t)], linewidth=0.5)
    ax[0].plot(df[('Close', t)], linewidth=1.5)
    ax[0].plot(df[('Stoploss', t)], linewidth=1.5)
    ax[1].plot(df[('Strat', t)], linewidth=0.5)
    ax[2].plot(df[('Percent_return', 'Total')].cumsum(), linewidth=1)
    ax[2].plot(df[('Strat_return', t)].cumsum(), linewidth=1, color='r')
    #ax[0].plot(df[('Strat_return', t)].cumsum(), linewidth=1, color='r')
r1=df[('Strat_return', tickers[0])].sum()
r2=df[('Percent_return', tickers[0])].sum()
print(f'Strat return SPY = {r1}')
print(f'Hold return SPY = {r2}')
end_time = datetime.datetime.now().time()
print(f'Time when finished:                   {end_time}\n')
print(df.index.max())


###NOTES
#For some reason not all data is used, 904 ipv 1004   ->  Maar laatste datum is wel incl.

01:21:22.880875
1005


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Strat return SPY = 0.36480040302028505
Hold return SPY = 0.5041826175961925
Time when finished:                   01:21:27.126419

1004


In [233]:
#https://stackoverflow.com/questions/19124601/pretty-print-an-entire-pandas-series-dataframe
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df)   

Attributes index       Date   Adj Close       Close        High         Low  \
Symbols                             SPY         SPY         SPY         SPY   
0              1 2016-01-05  182.822327  201.360001  201.899994  200.050003   
1              2 2016-01-06  180.516174  198.820007  200.059998  197.600006   
2              3 2016-01-07  176.185333  194.050003  197.440002  193.589996   
3              4 2016-01-08  174.251419  191.919998  195.850006  191.580002   
4              5 2016-01-11  174.423950  192.110001  193.410004  189.820007   
5              6 2016-01-12  175.831238  193.660004  194.550003  191.139999   
6              7 2016-01-13  171.445892  188.830002  194.860001  188.380005   
7              8 2016-01-14  174.260468  191.929993  193.259995  187.660004   
8              9 2016-01-15  170.519791  187.809998  188.759995  185.520004   
9             10 2016-01-19  170.746765  188.059998  190.110001  186.199997   
10            11 2016-01-20  168.558640  185.649994  187.500000  181.020004   
11            12 2016-01-21  169.502914  186.690002  188.869995  184.639999   
12            13 2016-01-22  172.980286  190.520004  190.759995  188.880005   
13            14 2016-01-25  170.365448  187.639999  190.149994  187.410004   
14            15 2016-01-26  172.689743  190.199997  190.529999  188.020004   
15            16 2016-01-27  170.810318  188.130005  191.559998  187.059998   
16            17 2016-01-28  171.700104  189.110001  190.199997  187.160004   
17            18 2016-01-29  175.885727  193.720001  193.880005  189.880005   
18            19 2016-02-01  175.822144  193.649994  194.580002  191.839996   
19            20 2016-02-02  172.653442  190.160004  191.970001  189.539993   
20            21 2016-02-03  173.688477  191.300003  191.779999  187.100006   
21            22 2016-02-04  173.960876  191.600006  192.750000  189.960007   
22            23 2016-02-05  170.646927  187.949997  191.669998  187.199997   
23            24 2016-02-08  168.349808  185.419998  186.119995  182.800003   
24            25 2016-02-09  168.358871  185.429993  186.940002  183.199997   
25            26 2016-02-10  168.213638  185.270004  188.339996  185.119995   
26            27 2016-02-11  166.025482  182.860001  184.100006  181.089996   
27            28 2016-02-12  169.448425  186.630005  186.649994  183.960007   
28            29 2016-02-16  172.308426  189.779999  189.809998  187.630005   
29            30 2016-02-17  175.123047  192.880005  193.320007  191.009995   
30            31 2016-02-18  174.405762  192.089996  193.270004  191.720001   
31            32 2016-02-19  174.324066  192.000000  192.179993  190.449997   
32            33 2016-02-22  176.848099  194.779999  194.949997  193.789993   
33            34 2016-02-23  174.614578  192.320007  194.320007  192.179993   
34            35 2016-02-24  175.413559  193.199997  193.529999  189.320007   
35            36 2016-02-25  177.538147  195.539993  195.550003  192.830002   
36            37 2016-02-26  177.129562  195.089996  196.679993  194.899994   
37            38 2016-02-29  175.740417  193.559998  196.229996  193.330002   
38            39 2016-03-01  179.871582  198.110001  198.210007  194.449997   
39            40 2016-03-02  180.679596  199.000000  199.059998  197.250000   
40            41 2016-03-03  181.387802  199.779999  199.800003  198.110001   
41            42 2016-03-04  181.977966  200.429993  201.350006  199.029999   
42            43 2016-03-07  182.123245  200.589996  201.070007  199.250000   
43            44 2016-03-08  180.134842  198.399994  199.919998  198.210007   
44            45 2016-03-09  181.024643  199.380005  199.789993  198.429993   
45            46 2016-03-10  181.169907  199.539993  201.070007  197.380005   
46            47 2016-03-11  184.093460  202.759995  202.809998  199.520004   
47            48 2016-03-14  183.857391  202.500000  203.039993  201.770004   
48            49 2016-03-15  183.557770  202.16999

In [71]:
tickers = ['SPY', 'AAPL', 'MSFT', 'PYPL', 'ISRG', 'GM', 'INTC', 'PANW']
tickers = ['SPY']
startd = datetime.datetime(2016, 1, 1) 
#endd_1 = datetime.datetime(2019, 1, 1)
endd_2 = datetime.datetime(2020, 1, 1)
df = data.DataReader(tickers, 'yahoo', startd, endd_2).reset_index()
for t in tickers:
    df[('Percent_return', t)] = df[('Close', t)].pct_change()
    df[('Percent_return', 'Total')] = df[('Close', t)].pct_change()

    
#Ehlers Simple Decycler (Vertaald van adneretaal)
#Bronnen: https://www.tradingview.com/script/ZuIZPR4q-Ehlers-Simple-Decycler/
##2: https://www.prorealcode.com/prorealtime-indicators/ehlers-simple-decycler/
##3 voor vertalen: https://www.tradingview.com/pine-script-docs/en/v4/Quickstart_guide.html#execution-model-of-pine-scripts
##4 "" history reference ioerator: https://www.tradingview.com/pine-script-docs/en/v4/language/Operators.html#History_Referencing_Operator_.28Square_Brackets_.5B.5D.29

for t in tickers:
    #hp = highpass
    hpLength = 125
    upper = 0.5
    lower = 0.5
    src = df[('Close', t)]

    alphaArg = 2*np.pi / (hpLength*np.sqrt(2))
    df[('Alpha', t)] = 0        
    df[('Hp', t)] = 0   
    df[('Decycler', t)] = 0  
    for i in range(125, len(df)):
        #Hp filter
        if np.cos(alphaArg) != 0:
            df.loc[i, ('Alpha', t)] = (np.cos(alphaArg)+np.sin(alphaArg)-1) / np.cos(alphaArg)
        else:
            df.loc[i, ('Alpha', t)] = df.loc[i-1, ('Alpha', t)]
        #Decycling::: src = the input!
        df.loc[i, ('Hp', t)] = np.square(1-(df.loc[i, ('Alpha', t)]/2)) * (src[i]-2 * src[i-1]+src[i-2]) + 2 * (1-df.loc[i, ('Alpha', t)]) * df.loc[i-1, ('Hp', t)] - np.square(1-df.loc[i, ('Alpha', t)]) * df.loc[i-2, ('Hp', t)]
        df.loc[i, ('Decycler', t)] = src[i] - df.loc[i, ('Hp', t)]
        if df.loc[i, ('Decycler', t)] >= df.loc[i-1, ('Decycler', t)]:
            df.loc[i, ('Decycler_signal', t)] = 1
        else:
            df.loc[i, ('Decycler_signal', t)] = 0
    
for t in tickers:
    fig, ax = plt.subplots(2, sharex=True, figsize=(10,7), gridspec_kw={'height_ratios':[2.5, 0.5], 'hspace':0.02})
    ax[0].plot(df.loc[125:, ('Decycler', t)], linewidth=1.5)
    ax[0].plot(df.loc[125:, ('Close', t)], linewidth=1.5)
    ax[1].plot(df.loc[125:, ('Decycler_signal', t)], linewidth=1.5)
    
display(df)   

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Attributes       Date   Adj Close       Close        High         Low  \
Symbols                       SPY         SPY         SPY         SPY   
0          2016-01-04  182.513626  201.020004  201.029999  198.589996   
1          2016-01-05  182.822327  201.360001  201.899994  200.050003   
2          2016-01-06  180.516174  198.820007  200.059998  197.600006   
3          2016-01-07  176.185333  194.050003  197.440002  193.589996   
4          2016-01-08  174.251419  191.919998  195.850006  191.580002   
...               ...         ...         ...         ...         ...   
1001       2019-12-24  315.341095  321.230011  321.519989  320.899994   
1002       2019-12-26  317.019714  322.940002  322.950012  321.640015   
1003       2019-12-27  316.941162  322.859985  323.799988  322.279999   
1004       2019-12-30  315.193817  321.079987  323.100006  320.549988   
1005       2019-12-31  315.959503  321.859985  322.130005  320.149994   

Attributes        Open       Volume Percent_return               Alpha  \
Symbols            SPY          SPY            SPY     Total       SPY   
0           200.490005  222353500.0            NaN       NaN  0.000000   
1           201.399994  110845800.0       0.001691  0.001691  0.000000   
2           198.339996  152112600.0      -0.012614 -0.012614  0.000000   
3           195.330002  213436100.0      -0.023992 -0.023992  0.000000   
4           195.190002  209817200.0      -0.010977 -0.010977  0.000000   
...                ...          ...            ...       ...       ...   
1001        321.470001   20270000.0       0.000031  0.000031  0.034926   
1002        321.649994   30911200.0       0.005323  0.005323  0.034926   
1003        323.739990   42528800.0      -0.000248 -0.000248  0.034926   
1004        322.950012   49729100.0      -0.005513 -0.005513  0.034926   
1005        320.529999   57077300.0       0.002429  0.002429  0.034926   

Attributes        Hp    Decycler Decycler_signal  
Symbols          SPY         SPY             SPY  
0           0.000000    0.000000             NaN  
1           0.000000    0.000000             NaN  
2           0.000000    0.000000             NaN  
3           0.000000    0.000000             NaN  
4           0.000000    0.000000             NaN  
...              ...         ...             ...  
1001        3.359661  317.870350             1.0  
1002        4.494857  318.445145             1.0  
1003        3.818623  319.041362             1.0  
1004        1.543016  319.536970             1.0  
1005        1.893075  319.966911             1.0  

[1006 rows x 13 columns]